In [28]:
from sklearn.datasets import load_iris  
import numpy as np
from IrisFisher import softmax, softmax_grad, logloss, sigmoid

In [29]:
data = load_iris()

def UnisonShuffle(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def random_init(size, left = -1., right = 1.):
    return np.random.random(size = size) * (right + left) + left

def net_init_1(params):
    # params["L0"] = np.zeros((1, 4))
    # 
    # params["W0"] = random_init((4, 4))
    # params['b0'] = random_init((1, 4))
    # 
    # params["L1"] = np.zeros((1, 4))
    params["P1"] = np.zeros((1, 4))
    
    params["W1"] = random_init((4, 3))
    params['b1'] = random_init((1, 3))
    
    params["L2"] = np.zeros((1, 3))
    params["P2"] = np.zeros((1, 3))
    
    params["E"] = np.zeros((1,1))
    
def check_shape_1(X, y):
    if X.shape !=  (1, 4):
        raise ValueError("X : Expect shape (1, 4), given {0}".format(X.shape))
    if y.shape !=  (1, 3):
        raise ValueError("X : Expect shape (1, 3), given {0}".format(y.shape))
    

In [30]:
def forward_prop_1(X, y, params):
    check_shape_1(X, y)
    
    # params["L0"] = X
    # 
    # params["L1"] = np.dot(params["L0"], params["W0"]) + params["b0"]
    # params["P1"] = sigmoid(params["L1"])
    
    params["P1"] = X
    
    params["L2"] = np.dot(params["P1"], params["W1"]) + params["b1"]
    params["P2"] = softmax(params["L2"])
    
    params["E"] = logloss(params["P2"], y)
    
def back_prop_1(X, y, params, giper_params):
    check_shape_1(X, y)
    
    params["dE/dP2"] = - y / params["P2"] # 1x3
    params["dP2/dL2"] = softmax_grad(params["P2"]) # 3x3
    params["dE/dL2"] = np.dot(params["dE/dP2"], params["dP2/dL2"]) # 1x3
    params["dE/dW1"] = np.dot(params["P1"].T, params["dE/dL2"]) #4x3
    params["dE/db1"] = params["dE/dL2"]
    
    # params["dE/dP1"] = np.dot(params["dE/dL2"], params["W1"].T)# 1x4
    # params["dP1/dL1"] = params["P1"] * (1 - params["P1"]) #1x4
    # params["dE/dL1"] = params["dE/dP1"] * params["dP1/dL1"] # 1x4
    # # params["dE/dL1"] = np.dot(params["dE/dL2"], params["W1"].T)
    # params["dE/dW0"] = np.dot(params["L0"].T, params["dE/dL1"]) # 4x4
    # params["dE/db0"] = params["dE/dL1"]
    
    params["W1"] -= giper_params["learning rate"] * params["dE/dW1"]
    params["b1"] -= giper_params["learning rate"] * params["dE/db1"]
    
    # params["W0"] -= giper_params["learning rate"] * params["dE/dW0"]
    # params["b0"] -= giper_params["learning rate"] * params["dE/db0"]
    

In [31]:
X = data["data"]
y_lables = data["target"]
y = np.zeros((len(y_lables), 3))
for i, a in enumerate(y_lables):
    y[i][a] = 1

# np.random.seed(179)
X, y = UnisonShuffle(X, y)
X = X.reshape(-1, 1, 4)
y = y.reshape(-1, 1, 3)
params = dict()
giper_params = dict()
giper_params["learning rate"] = 0.1
lenX = len(y)


net_init_1(params)
    

In [32]:
for i in range(lenX * 200):
    X_cur = X[i % lenX]
    y_cur = y[i % lenX]
    forward_prop_1(X_cur, y_cur, params)
    back_prop_1(X_cur, y_cur, params, giper_params)

In [33]:
error_counter = 0


for iter, item in enumerate(y):
    X_cur = X[iter % lenX]
    y_cur = y[iter % lenX]
    forward_prop_1(X_cur, y_cur, params)
    if np.argmax(y_cur) != np.argmax(params["P2"]):
        error_counter += 1
    print(np.argmax(y_cur), np.argmax(params["P2"]))
    print(params["P2"])
print(error_counter)



0 0
[[9.99999839e-01 1.60743192e-07 1.87981366e-49]]
1 1
[[1.31338997e-04 9.99868606e-01 5.46618067e-08]]
0 0
[[9.99999812e-01 1.87918741e-07 1.00662505e-50]]
1 1
[[4.18263021e-04 9.99567453e-01 1.42835498e-05]]
2 2
[[1.89102332e-12 5.61665196e-07 9.99999438e-01]]
0 0
[[9.99999992e-01 7.98255730e-09 6.70955209e-56]]
1 1
[[5.95093479e-06 9.99431541e-01 5.62507723e-04]]
1 1
[[1.60313524e-04 9.99701259e-01 1.38427297e-04]]
2 2
[[1.45246648e-19 1.25587778e-11 1.00000000e+00]]
0 0
[[9.99999945e-01 5.54301984e-08 6.57163796e-55]]
2 2
[[1.57561323e-12 3.83138842e-07 9.99999617e-01]]
0 0
[[9.99985117e-01 1.48834590e-05 1.99933792e-45]]
2 2
[[2.17620613e-17 1.31893113e-10 1.00000000e+00]]
1 1
[[2.23635407e-05 9.99977197e-01 4.39422408e-07]]
2 2
[[3.70800876e-15 1.00833850e-08 9.99999990e-01]]
0 0
[[9.99999920e-01 8.04414346e-08 4.17909916e-51]]
1 1
[[3.55412741e-04 9.99636748e-01 7.83923498e-06]]
2 2
[[2.00087852e-15 8.79250584e-09 9.99999991e-01]]
1 1
[[1.45702661e-03 9.98542956e-01 1.72127319